# Roadmap

1. Get data
2. Create train-test split
3. Clean training set (write functions): missing values, text, categorical attributes, scaling
4. Select models and scoring metrics, then train
5. Compare them: clean test set, make predictions, score
6. Fine-tune models

In [1]:
%load_ext autoreload
%autoreload 2
import autoreload

## 1. Load data

In [6]:
from functions import load_data
data_df = load_data('data/time_series.xlsx')
data_df.info()
data_df.tail()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 292 entries, 2012-04-08 to 2017-11-05
Columns: 1833 entries, 012 to TRUHONE
dtypes: int64(1833)
memory usage: 4.1 MB


012  017  03008944ST-1  03008944ST-3  0300ST1550-1  0300ST15X9-1  \
EntDate                                                                        
2017-10-08    0    0             0             0             0             0   
2017-10-15    0    0             0             0             0             0   
2017-10-22    0    0             0             0             0             0   
2017-10-29    0    0             0             0             0             0   
2017-11-05    0    0             0             0             0             0   

            0300ST15X9-2  0300ST15X9-3  0300ST1605-1  0300ST1605-2   ...     \
EntDate                                                              ...      
2017-10-08             0             0             0             0   ...      
2017-10-15             0             0             0             0   ...      
2017-10-22             0             0             0             0   ...      
2017-10-29             0             0             0             0   ...      
2017-11-05             0             0             0             0   ...      

            9920-2  9920-3  9920-4  9920-5  9920-6  9920-7  9997-25  HW220D15  \
EntDate                                                                         
2017-10-08       0       3       0       0       0      13        0         0   
2017-10-15       0       0       0       0       0       0        0         0   
2017-10-22       0       0       0       0       0       0        0         0   
2017-10-29       0       0       0       0       0       0        2         0   
2017-11-05       0       0       0       0       0       0        0         0   

            HW240DIA  TRUHONE  
EntDate                        
2017-10-08         0        0  
2017-10-15         0        0  
2017-10-22         0        0  
2017-10-29         0        0  
2017-11-05         0        0  

[5 rows x 1833 columns]

## 2. Train-Test Split

In [ ]:
# splitting into training and testing sets setting aside last year for testing
from functions import ts_train_test_split

train_df, test_df = ts_train_test_split(data_df, 52)
train_df.head()

## Train-Test Split -- alternative

In [ ]:
# now we will make three datasets using 33-33-34
X = data_df.values
train_size1 = int(len(X) * 0.33)
train_size2 = int(len(X) * 0.66)
pretrain, train, test = X[0:train_size1], X[train_size1:train_size2], X[train_size2:len(X)]
print('Observations: %d' % (len(X)))
print('Initializing Observations: %d' % (len(pretrain)))
print('Training Observations: %d' % (len(train)))
print('Testing Observations: %d' % (len(test)))
# made train_df and test_df (the latter to be used later)
pretrain_df = data_df[0:train_size1]
train_df = data_df[train_size1:train_size2]
test_df = data_df[train_size2:292]
train_df.head()

## Select single item for forecasting

In [ ]:
# pick one item
test = train_df['9920-2']
test.head()

In [ ]:
# plot one item
from functions import plot_train_test
plot_train_test(train_df, test_df, '9920-2')

## 3. Clean Training Set

In [ ]:
# make a copy first

In [ ]:
# make transformations pipeline (first applied to train, then apply to test)
# luckily, our data has no null values, no categorical/text values
# however, to get the initial messy excel sheet into timeseries format was not easy
# describe

## 4. Select Models and Scoring Metrics

We are comparing different forecasting models on the same data, so we can use scale-dependent errors because our single dataset only has one scale. We will use two metrics: <br>
* MAD (also called MAE): $\frac{|A_t-F_t|}{n}$ <br>
* RMSE (root mean squared error): $\sqrt{\frac{(A_t-F_t)^2}{n}}$ <br>

For baseline, we will use:
* naive forecast ("only yesterday matters")

Then we will try two simple forecasts:
* cumulative ("history matters")
* moving average ("I select how much matters")

We will also try:
* ARIMA
* exponential smoothing

### Make Forecasts

In [ ]:
# make a df to store all our predictions
from functions import make_copy_df
y_hat = make_copy_df(test_df, '9920-2')

### Naive Forecast (Baseline)

In [ ]:
# FIX LATER
import numpy as np

dd= np.asarray(train_df['9920-2'])
y_hat['naive'] = dd[len(dd)-1] # this line of code is for one-time forecast

In [ ]:
# plot
from functions import plot_time_series
plot_time_series(train_df, test_df, '9920-2', y_hat, 'naive', 'Naive Forecast')

In [ ]:
# calculate RMSE
from functions import RMSE

RMSE(test_df, '9920-2', y_hat, 'naive')

In [ ]:
# calculate MAE
from functions import MAE

MAE(test_df, '9920-2', y_hat, 'naive')

### Cumulative

In [ ]:
from functions import cumulative
y_hat['cumulative'] = cumulative(train_df['9920-2'])
plot_time_series(train_df, test_df, '9920-2', y_hat, 'cumulative', 'Cumulative Forecast')

In [ ]:
# calculate RMSE
from functions import RMSE

RMSE(test_df,'9920-2', y_hat, 'cumulative')

In [ ]:
# calculate MAE
from functions import MAE

MAE(test_df,'9920-2', y_hat, 'cumulative')

### Moving Average

In [ ]:
# forecasts for a quarter year do better than half year or full year
from functions import moving_average
y_hat['moving_avg'] = moving_average(train_df['9920-2'], m=13)

plot_time_series(train_df, test_df, '9920-2', y_hat, 'moving_avg', 'Moving Average Forecast')

In [ ]:
# calculate RMSE
from functions import RMSE

RMSE(test_df,'9920-2', y_hat, 'moving_avg')

In [ ]:
# calculate MAE
from functions import MAE

MAE(test_df,'9920-2', y_hat, 'moving_avg')

### S/ARIMA

In [ ]:
# gridsearch with pyramid
from pyramid.arima import ARIMA

fit = ARIMA(order=(1, 1, 0), seasonal_order=(1, 0, 0, 12)).fit(y=train_df['9920-2'])

from pyramid.arima import auto_arima

stepwise_fit = auto_arima(train_df['9920-2'], start_p=0, start_q=0, max_p=6, max_q=6, m=12,
                          start_P=0, seasonal=True, d=1, D=1, trace=True,
                          error_action='ignore',  # don't want to know if an order does not work
                          suppress_warnings=True,  # don't want convergence warnings
                          stepwise=True)  # set to stepwise

stepwise_fit.summary()

In [ ]:
# fit with SARIMAX
from statsmodels.tsa.statespace.sarimax import SARIMAX

fit1 = SARIMAX(train_df['9920-2'], order=(0, 1, 1), seasonal_order=(0,1,2,12), freq='W').fit()
y_hat['SARIMA'] = fit1.predict(start="2016-11-06", end="2017-11-05", dynamic=True, typ='levels')

# plot
plot_time_series(train_df, test_df, '9920-2', y_hat, 'SARIMA', 'SARIMA Forecast')

In [ ]:
# calculate RMSE
from functions import RMSE

RMSE(test_df,'9920-2', y_hat, 'SARIMA')

### Exponential Smoothing

In [ ]:
# let's decompose it first to see trend, seasonality
from functions import decompose_timeseries

decompose_timeseries(train_df['9920-2'], 'additive')

In [ ]:
# Double Exponential Smoothing with Statsmodels
# no trend, just seasonality (multiplicative), no damping
from statsmodels.tsa.api import ExponentialSmoothing

fit1 = ExponentialSmoothing(np.asarray(train_df['9920-2']), seasonal_periods=10, trend=None, seasonal='additive').fit(smoothing_level=0.51, smoothing_seasonal=0.1)
y_hat['DES'] = fit1.forecast(len(test_df))

# plot
plot_time_series(train_df, test_df, '9920-2', y_hat, 'DES', 'Double ES Forecast')

In [ ]:
# calculate RMSE
from functions import RMSE

RMSE(test_df,'9920-2', y_hat, 'DES')

In [ ]:
import numpy as np
# here we have trend and seasonality, so we will use Holt-Winters with Statsmodels
# additive trend and seasonality, no trend damping, seasonal periods=10
from statsmodels.tsa.api import ExponentialSmoothing

fit1 = ExponentialSmoothing(np.asarray(train_df['9920-2']), seasonal_periods=10, 
                            trend='additive', seasonal='additive').fit(smoothing_level=0.51, 
                                                             smoothing_slope=0.015,
                                                             smoothing_seasonal=0.1)
y_hat['Holt_Winter'] = fit1.forecast(len(test_df))

plot_time_series(train_df, test_df, '9920-2', y_hat, 'Holt_Winter', 'Holt_Winter Forecast')

In [ ]:
# calculate RMSE
from functions import RMSE

RMSE(test_df,'9920-2', y_hat, 'Holt_Winter')

In [ ]:
# calculate MAE
from functions import MAE

MAE(test_df,'9920-2', y_hat, 'Holt_Winter')

### issue 1: lack of gridsearch capability

In [ ]:
# try statsmodels HW with GridSearch -- doesn't work

In [3]:
# create homemade GridSearch for statsmodel HW -- WORK NEEDED
# homemade gridsearch for ARIMA

from sklearn.metrics import mean_squared_error
from numpy import sqrt

# evaluate HW model for a given order (p,d,q)
def evaluate_holtwinters_model(X, a, b, g):
    # prepare training dataset
    train_size = int(len(X) * 0.66)
    train, test = X[0:train_size], X[train_size:]
    history = [x for x in train]
    # make predictions
    predictions = list()
    for t in range(len(test)):
        #model = ARIMA(train, order=arima_order)
        model = ExponentialSmoothing(train, seasonal_periods=10, 
                            trend='add', seasonal='add')
        model_fit = model.fit(disp=0, smoothing_level=a, 
                                    smoothing_slope=b,
                                    smoothing_seasonal=g)
        yhat = model_fit.forecast()[0]
        predictions.append(yhat)
        history.append(test[t])
    # calculate out of sample error
    error = sqrt(mean_squared_error(test, predictions))
    return error

import warnings
warnings.filterwarnings("ignore")

def evaluate_models(dataset, a_values, b_values, g_values):
    dataset = dataset.astype('float32')
    best_score, best_cfg = float("inf"), None
    for a in a_values:
        for b in b_values:
            for g in g_values:
                smoothing_level, smoothing_slope, smoothing_seasonal=a, b, g
                try:
                    mse = evaluate_holtwinters_model(dataset, smoothing_level, smoothing_slope, smoothing_seasonal)
                    if mse < best_score:
                        best_score, best_cfg = mse, smoothing_level, smoothing_slope, smoothing_seasonal
                    print('HW%s RMSE=%.3f' % (smoothing_level, smoothing_slope, smoothing_seasonal,mse))
                except:
                    continue
    print('Best HW%s RMSE=%.3f' % (best_cfg, best_score))

In [4]:
dataset = data_df['9920-2']
a_values = [0.02, 0.1, 0.19, 0.35, 0.51]
b_values = [0.005, 0.029, 0.053, 0.094, 0.135, 0.176]
g_values = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5]
evaluate_models(dataset, a_values, b_values, g_values)

Best HWNone RMSE=inf


### Exponential Smoothing with Homebrewed Functions

In [ ]:
from exponential_smoothing import initial_trend, initial_seasonal_components, triple_exponential_smoothing

In [ ]:
predictions = triple_exponential_smoothing(train_df['9920-2'], slen=10, alpha=0.51, beta=0.015, gamma=0.1, n_preds=52)
y_hat['HW_new'] = predictions[-52:]

In [ ]:
# calculate RMSE
from functions import RMSE

RMSE(test_df, '9920-2', y_hat, 'HW_new')

### Exponential Smoothing with Homebrewed Class

In [ ]:
# additive trend and seasonality, no trend damping, seasonal periods=10
from HoltWinters_class2 import HoltWinters

In [ ]:
model = HoltWinters(alpha=0.51, beta=0.015, gamma=0.1, n_preds=52)
model.fit(train_df['9920-2'])

preds = model.predict(train_df['9920-2'])
y_hat['Holt_Winter'] = preds[-52:]

from functions import plot_time_series
plot_time_series(train_df, test_df, '9920-2', y_hat, 'Holt_Winter', 'Holt_Winter Forecast')

In [ ]:
# calculate RMSE
model.scorer(train_df['9920-2'], test_df['9920-2'])

### issue 1: lack of gridsearch capability

In [ ]:
# tried running checkestimator -- failed
# tried gridsearch with custom scoring function -- failed
# tried gridsearch after removing regressormixin -- failed
# tried gridsearch without score() method inside class -- failed
# tried gridsearch after removing regressormixin with custom scoring function -- failed

### issue 2: initialization

### Prophet

In [ ]:
# imports 
from fbprophet import Prophet

In [ ]:
# make a copy of the dataframe for Prophet transformations
from functions import make_copy_df
prophet_df = make_copy_df(train_df, '9920-2')

In [ ]:
# rename variables (prophet requires the variable names in the time series to be
# y for target and ds for Datetime)
from functions import rename_columns
rename_columns(prophet_df, '9920-2')
prophet_df.head()

In [ ]:
# FIX LATER
# set the uncertainty interval to 95% (the Prophet default is 80%)
my_model = Prophet(growth='linear', interval_width=0.95, weekly_seasonality=True)
my_model.fit(prophet_df)

In [ ]:
# FIX LATER
future_dates = my_model.make_future_dataframe(periods=52, freq='W')
forecast = my_model.predict(future_dates)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
# FIX LATER
forecast.head()
forecast = forecast.set_index('ds')
forecast_slice=forecast[240:292]
forecast_df = forecast_slice["yhat"]
forecast_df.head()

In [ ]:
# plot
plot_time_series(train_df, test_df, '9920-2', forecast_slice, 'yhat', 'Prophet Forecast')

In [ ]:
# calculate RMSE
from functions import RMSE

RMSE(test_df, '9920-2', forecast_slice, 'yhat')

## Resources

Great basic blogs on time series and forecasting methods: https://medium.com/@josemarcialportilla/using-python-and-auto-arima-to-forecast-seasonal-time-series-90877adff03c

On Prophet:
https://www.analyticsvidhya.com/blog/2018/05/generate-accurate-forecasts-facebook-prophet-python-r/
https://research.fb.com/prophet-forecasting-at-scale/
(Plus see 